In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
def bureau_engineering():
    description = pd.read_csv('../Input/HomeCredit_columns_description.csv', encoding = 'latin1')
    application = pd.read_csv('../Input/application_engineered.csv')
    bureau = pd.read_csv('../Input/bureau.csv')
    bureau[bureau['SK_ID_CURR']==215354]
    
    
    bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
    bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

    groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])
            
    description[description['Row'] == 'DAYS_CREDIT'].Description.tolist()
    features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
            
    group_object = groupby_SK_ID_CURR['DAYS_CREDIT'].agg('count').reset_index()
    group_object.rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
            
    group_object = groupby_SK_ID_CURR['CREDIT_TYPE'].agg('nunique').reset_index()
    group_object.rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    features['bureau_average_of_past_loans_per_type'] = \
        features['bureau_number_of_past_loans'] / features['bureau_number_of_loan_types']
    
    group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('mean').reset_index()

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index()
    group_object.rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM'].agg('sum').reset_index()
    group_object.rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    features['bureau_debt_credit_ratio'] = \
    features['bureau_total_customer_debt'] / features['bureau_total_customer_credit']
    
    
    group_object  = groupby_SK_ID_CURR['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index()
    group_object.rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    features['bureau_overdue_debt_ratio'] = \
    features['bureau_total_customer_overdue'] / features['bureau_total_customer_debt']
    
    group_object = groupby_SK_ID_CURR['CNT_CREDIT_PROLONG'].agg('sum').reset_index()
    group_object.rename(index=str, columns={'CNT_CREDIT_PROLONG': 'bureau_average_creditdays_prolonged'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = groupby_SK_ID_CURR['bureau_credit_enddate_binary'].agg('mean').reset_index()
    group_object.rename(index=str, columns={'bureau_credit_enddate_binary': 'bureau_credit_enddate_percentage'},inplace=True)

    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    bureau_ONE = features[features['SK_ID_CURR']==215354]
    
    application = application.merge(features,
                                left_on=['SK_ID_CURR'],
                                right_on=['SK_ID_CURR'],
                                how='left',
                                validate='one_to_one')
    
    BUREAU_AGGREGATION_RECIPIES = [('CREDIT_TYPE', 'count'), ('CREDIT_ACTIVE', 'size')]
    for agg in ['mean', 'min', 'max', 'sum', 'var']:
        for select in ['AMT_ANNUITY',
                       'AMT_CREDIT_SUM',
                       'AMT_CREDIT_SUM_DEBT',
                       'AMT_CREDIT_SUM_LIMIT',
                       'AMT_CREDIT_SUM_OVERDUE',
                       'AMT_CREDIT_MAX_OVERDUE',
                       'CNT_CREDIT_PROLONG',
                       'CREDIT_DAY_OVERDUE',
                       'DAYS_CREDIT',
                       'DAYS_CREDIT_ENDDATE',
                       'DAYS_CREDIT_UPDATE'
                           ]:
            BUREAU_AGGREGATION_RECIPIES.append((select, agg))
    BUREAU_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], BUREAU_AGGREGATION_RECIPIES)]
    
    groupby_aggregate_names = []
    for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
        group_object = bureau.groupby(groupby_cols)
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
            application = application.merge(group_object[select]
                                  .agg(agg)
                                  .reset_index()
                                  .rename(index=str,
                                          columns={select: groupby_aggregate_name})
                                  [groupby_cols + [groupby_aggregate_name]],
                                  on=groupby_cols,
                                  how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)
            
    return application

In [3]:
application = bureau_engineering()

In [4]:
application.shape

(356255, 329)

In [5]:
application.head()

,Unnamed: 0,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,...,SK_ID_CURR_var_AMT_CREDIT_SUM,SK_ID_CURR_var_AMT_CREDIT_SUM_DEBT,SK_ID_CURR_var_AMT_CREDIT_SUM_LIMIT,SK_ID_CURR_var_AMT_CREDIT_SUM_OVERDUE,SK_ID_CURR_var_AMT_CREDIT_MAX_OVERDUE,SK_ID_CURR_var_CNT_CREDIT_PROLONG,SK_ID_CURR_var_CREDIT_DAY_OVERDUE,SK_ID_CURR_var_DAYS_CREDIT,SK_ID_CURR_var_DAYS_CREDIT_ENDDATE,SK_ID_CURR_var_DAYS_CREDIT_UPDATE
0,0,0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,...,2.133807e+10,1.208166e+10,2.558171e+08,0.0,5.584936e+06,0.0,0.0,186150.000000,5.890424e+05,268865.553571
1,1,1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,...,1.385846e+11,0.000000e+00,1.640250e+11,0.0,0.000000e+00,0.0,0.0,827783.583333,2.228364e+06,824562.000000
2,2,2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,...,7.144200e+02,0.000000e+00,0.000000e+00,0.0,NaN,0.0,0.0,421362.000000,2.268450e+04,45000.000000
3,3,3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
application.to_csv('../Input/app_bureau_engineered.csv')